In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import mwparserfromhell
import urllib.request
import json
import re
import numpy as np
import pickle
from tqdm.notebook import tqdm

url = "https://wowpedia.fandom.com/wiki/Major_characters"
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")



In [8]:
characters = []

games = ['WC1','WC2','WC2X','WC3','WC3X','WoW','WoWBC','WoWWotLK','WoWC','WoWMoP','WoWWoD','WoWL','WoWBfA','WoWS','WoWD']

for s in soup.find_all('p'):
    text = s.text
    if "—" in text:
        
        d = {}
        
        t = text.replace("\n", "").split("—")

        name = t[0].strip().replace(" ", "_") #.replace(" ", "")

        expansions = t[1].replace(" ", "").split(",")
        expansions = [1 if i in expansions else 0 for i in games]
        
        if 1 in expansions:
            d['name'] = name
            for i,g in enumerate(games):
                d[g] = expansions[i]
                
            characters.append(d)


In [14]:
with open('df.pickle', 'rb') as f:
    df = pickle.load(f)
# df = df.reset_index(drop=True)
df

,Name,Race,Affiliation
0,Terokk,Arakkoa,0
1,Talon_King_Ikiss,Arakkoa,Other
2,Archimonde,Man'ari eredar,Other
3,Kil'jaeden,Man'ari eredar,Other
4,Mannoroth,Annihilan,Other
...,...,...,...
2520,the_Jade_Warlord,Mogu,0
2521,the_Jailer,Eternal One,Other
2522,the_Monkey_King,Hozen,Other
2523,the_Stonewright,Venthyr,Other


In [10]:
names = list(df['Name'])
itemus = names[0]

url = f"https://wowpedia.fandom.com/api.php?action=query&prop=revisions&titles={itemus}&rvslots=*&rvprop=content&formatversion=2&format=json"
response = requests.get(url)
 
itemdetail = response.json()
t = itemdetail['query']['pages'][0]['revisions'][0]['slots']['main']['content']

print(t)

{{Foralt}}
{{npcbox
| name = Terokk
| image = Terokk.jpg
| titles = The Talon King
| gender = Male
| level = 72
| type = Elite
| race = Arakkoa
| creature = Undead
| faction = Combat
| aggro = {{aggro|-1|-1}}
| foccupation = Talon King of the Arakkoa, King of the Outcasts and Founder of Skettis
| status = Deceased (lore)<br/> Killable {{bc-inline}}
| location = [[Terokk's Rest]] in [[Skettis]], [[Terokkar Forest]]
| relatives = [[Lithic]] (daughter)
}}

'''Terokk''' was the greatest king of the [[arakkoa]]. Terokk was associated with the goddess [[Rukhmar]] and believed to be her incarnation, though this was not actually the case. Terokk was the arakkoa's greatest hero who was betrayed by his own allies and afflicted with the Curse of Sethe. Even in this state, however, he sought to aid his fellow refugees by building the city of [[Skettis]] with his own claws. However the curse soon began to drive him to madness, and he was sealed away by his Talonpriests in the hopes of rejuvenating 

In [14]:
def get_wikicode(t):

    response = requests.get(
         'https://wowpedia.fandom.com/api.php',
             params={
             'action': 'query',
             'format': 'json',
             'titles': '{}'.format(t),
             'prop': 'revisions',
             'rvprop': 'content',
         }
     ).json()


    page = next(iter(response['query']['pages'].values()))

    wikicode = page['revisions'][0]['*']
    
    return wikicode

In [59]:
from urllib.error import HTTPError

characters = []

for name in tqdm(list(df['Name'])):

    text = get_wikicode(name)

    WORD_RE = re.compile(r"\[{2}.*?\]{2}")
    for word in WORD_RE.findall(text):
        try:
            char = word.replace(" ", "_")[2:-2]
            url = "https://wowpedia.fandom.com/wiki/{}".format(char)
            html = urlopen(url).read()
            soup = BeautifulSoup(html, features="html.parser")
                
            table = soup.find_all('table')
            if len(table) > 0:

                text = table[0].text
                if "Gender" not in text:
                    pass
                else:
                    characters.append(char)
                
        except:
            pass
        
    
#     break

characters = list(np.unique(characters))
print(characters)    

UnicodeEncodeError: 'ascii' codec can't encode character '\u200e' in position 42: ordinal not in range(128)

In [5]:
df = df.iloc[0:272]

for name in list(df['Name']):
    url = "https://wowpedia.fandom.com/wiki/{}".format(name)
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")
    
    soup_a = soup.select("p > a")
    for a in soup_a:
        text = a['href']
        print(text)
    
    break
    
    

/wiki/Arakkoa
/wiki/Rukhmar
/wiki/Skettis
/wiki/Talon_King_Ikiss
/wiki/Kaliri
/wiki/Lower_City
/wiki/Dark_Portal
/wiki/Spires_of_Arak
/wiki/Lithic
/wiki/Warp_Stalker
/wiki/Wind_Serpent
/wiki/Colossus
/wiki/Pridelord_Karash
/wiki/Bloodmane_tribe
/wiki/Saberon
/wiki/Rukhmar
/wiki/Pridelord_Karash
/wiki/Skyreach
/wiki/Apexis
/wiki/Sethekk_Hollow
/wiki/Sethe
/wiki/Adherents_of_Rukhmar
/wiki/Anzu
/wiki/Dread_raven
/wiki/Void
/wiki/Arcane
/wiki/Talonpriest
/wiki/Skettis
/wiki/Arakkoa_Outcasts
/wiki/Talador
/wiki/Terokkar_Forest
/wiki/World_of_Warcraft:_The_Burning_Crusade
/wiki/Veil_Shienor
/wiki/Veil_Reskk
/wiki/Veil_Skith
/wiki/Darkstone_of_Terokk
/wiki/Veil_Rhaze
/wiki/Veil_Lithic
/wiki/Kaliri
/wiki/Veil_Shalas
/wiki/Sage
/wiki/Kirrik_the_Awakened
/wiki/Rilak_the_Redeemed
/wiki/Skettis_Exiles
/wiki/Shattrath_City
/wiki/Light
/wiki/Talon_King_Ikiss
/wiki/Sethekk
/wiki/Sethekk_Halls
/wiki/Sha%27tari_Skyguard
/wiki/Terokk%27s_Rest
/wiki/Hazzik
/wiki/Terokk%27s_Rest
/wiki/Ancient_Skull_Pile
/

In [24]:
parsed_wikicode

'Terokk was the greatest king of the arakkoa. Terokk was associated with the goddess Rukhmar and believed to be her incarnation, though this was not actually the case. Terokk was the arakkoa\'s greatest hero who was betrayed by his own allies and afflicted with the Curse of Sethe. Even in this state, however, he sought to aid his fellow refugees by building the city of Skettis with his own claws. However the curse soon began to drive him to madness, and he was sealed away by his Talonpriests in the hopes of rejuvenating him. He now waits in the shadows to be called back into the physical world and face the enemies of his people. \n\nThe heretical Talon King Ikiss claims he is Terokk returned. \n\nTerokk also has an avatar in the form of a gigantic kaliri, unleashed on Shattrath when all of his plans are thwarted by adventurers at the behest of the Skettis refugees of the Lower City. \n\nBiography\nOn Draenor\nthumb|left|Terokk doing battle with the Bloodmane saberon, before becoming cu

In [25]:
baseurl = "https://wowpedia.fandom.com/api.php?"
action = "action=query"
title = "titles={}"
content = "prop=revisions&rvprop=content"
dataformat ="format=json"

query = "{}{}&{}&{}&{}".format(baseurl, action, content, title.format(t), dataformat)
wikiresponse = urllib.request.urlopen(query)
wikidata = wikiresponse.read()
wikitext = wikidata.decode('utf-8')

In [26]:
wikitext

'{"batchcomplete":"","warnings":{"main":{"*":"Subscribe to the mediawiki-api-announce mailing list at <https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/> for notice of API deprecations and breaking changes."},"revisions":{"*":"Because \\"rvslots\\" was not specified, a legacy format has been used for the output. This format is deprecated, and in the future the new format will always be used."}},"query":{"pages":{"86839":{"pageid":86839,"ns":0,"title":"Terokk","revisions":[{"contentformat":"text/x-wiki","contentmodel":"wikitext","*":"{{Foralt}}\\n{{npcbox\\n| name = Terokk\\n| image = Terokk.jpg\\n| titles = The Talon King\\n| gender = Male\\n| level = 72\\n| type = Elite\\n| race = Arakkoa\\n| creature = Undead\\n| faction = Combat\\n| aggro = {{aggro|-1|-1}}\\n| foccupation = Talon King of the Arakkoa, King of the Outcasts and Founder of Skettis\\n| status = Deceased (lore)<br/> Killable {{bc-inline}}\\n| location = [[Terokk\'s Rest]] in [[Skettis

In [27]:
WORD_RE = re.compile(r"\[{2}.*?\]{2}")

for word in WORD_RE.findall(wikitext):
    word = word[2:-2].replace(" ", "_")
    print(word)

Terokk's_Rest
Skettis
Terokkar_Forest
Lithic
arakkoa
Rukhmar
Skettis
Talon_King_Ikiss
kaliri
Lower_City
File:Terrok_Battle.png|thumb|left|Terokk_doing_battle_with_the_Bloodmane_saberon,_before_becoming_cursed.
Image:Avatar_of_Terokk.jpg|thumb|The_Avatar_of_Terokk
Dark_Portal
World_of_Warcraft:_Chronicle_Volume_2
spires_of_Arak
Hidden_in_Plain_Sight
Lithic
Warp_Stalker|stalkers
Wind_Serpent|wind_serpents
Colossus|colossi
Adversarial_Blood
Pridelord_Karash
Bloodmane_tribe|Bloodmane
saberon
Rukhmar
The_Talon_King
Pridelord_Karash
Skyreach
Apexis
Sethekk_Hollow
Sethe
Adherents_of_Rukhmar
Anzu
dread_raven
The_Eye_of_Anzu
Void|dark
arcane
Talonpriest
Skettis
Arakkoa_Outcasts
Talador
Terokkar_Forest
Dreamcatcher
Beakbreaker_of_Terokk
Hazzik's_Bargain
Reshad
Terokk's_Fall
Isfar
Countdown_to_Doom
Veil_Shienor
Veil_Reskk
The_Eyes_of_Skettis
Veil_Skith
Darkstone_of_Terokk
Veil_Skith:_Darkstone_of_Terokk
Veil_Rhaze
Veil_Rhaze:_Unliving_Evil
Veil_Lithic
kaliri
Veil_Lithic:_Preemptive_Strike
Veil_Sh

Below we scrape the clean text from each characters page:

In [28]:
from tqdm import tqdm
text_list = []
for name in tqdm(df['Name']):
    response = requests.get(
         'https://wowpedia.fandom.com/api.php',
             params={
             'action': 'query',
             'format': 'json',
             'titles': '{}'.format(name),
             'prop': 'revisions',
             'rvprop': 'content',
         }
     ).json()


    page = next(iter(response['query']['pages'].values()))
    try:
        wikicode = page['revisions'][0]['*']
        parsed_wikicode = mwparserfromhell.parse(wikicode).strip_code()
        text_list.append(parsed_wikicode)
    except:
        text_list.append(" ")
    
df.insert(1,'Clean Text',text_list)


100%|██████████████████████████████████████████████████████████████████████████████| 1972/1972 [10:57<00:00,  3.00it/s]


In [29]:
df_backup = df

In [30]:
from tqdm import tqdm
alphabet = 'abcdefghijklmnopqrstuvwxyz'.upper()
c_list = []
for letter in tqdm(alphabet):
    url = "https://wowpedia.fandom.com/wiki/Category:Lore_characters?pagefrom={}".format(letter)
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")
    
    for i in soup.find_all('li'):
        text = i.find('a')['href']
        verboten = ['/wiki/Genealogy', '/wiki/Local_Sitemap']
        if text[0:5] == '/wiki' and ":" not in text and text not in verboten:
            c_list.append(text.replace("%27", "'").split("/wiki/")[1])

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:19<00:00,  1.31it/s]


In [33]:
#Find links for all characters:

baseurl = "https://wowpedia.fandom.com/api.php?"
action = "action=query"
title = "titles={}"
content = "prop=revisions&rvprop=content"
dataformat ="format=json"

WORD_RE = re.compile(r"\[{2}.*?\]{2}")

link_list=[]
for name in tqdm(df['Name']):
    query = "{}{}&{}&{}&{}".format(baseurl, action, content, title.format(name), dataformat)
    wikiresponse = urllib.request.urlopen(query)
    wikidata = wikiresponse.read()
    wikitext = wikidata.decode('utf-8')
    
    links = []
    for word in WORD_RE.findall(wikitext):
        word = word[2:-2].replace(" ", "_")
        if word in c_list or word in list(df['Name']) and word not in links:
            links.append(word)
    link_list.append(np.unique(links))

df.insert(2,'Out Links',link_list)

100%|██████████████████████████████████████████████████████████████████████████████| 1972/1972 [10:59<00:00,  2.99it/s]


In [34]:
df

,Name,Clean Text,Out Links
0,Terokk,Terokk was the greatest king of the arakkoa. T...,"[Anzu, Rukhmar, Sethe, Talon_King_Ikiss]"
1,Talon_King_Ikiss,"Talon King Ikiss (pronounced Eye-kiss), the bo...","[Anzu, Sethe, Terokk]"
2,Archimonde,Archimonde (pronounced ARK-i-mond) was one of ...,"[Alexstrasza, Azgalor, Cenarius, Grom_Hellscre..."
3,Kil'jaeden,Kil'jaeden (sometimes spelled as Kil'Jaeden)Ar...,"[Archimonde, Arthas_Menethil, Garrosh_Hellscre..."
4,Mannoroth,Mannoroth was the barbaric leader of the pit l...,"[Archimonde, Cenarius, Durotan, Garrosh_Hellsc..."
...,...,...,...
1967,Zur'ak_Firefist,Zur'ak Firefist is a troll shaman with the Ear...,"[Drek'Thar, Kettara_Bloodthirst, Krelna, Lenka..."
1968,Zuura,"left|thumb|Family tree.\nZuura, mate of Kelkar...","[Durotan, Garad, Kelkar, Rehze, Rhakish, Thrall]"
1969,Zuuzuu,Zuuzuu was a troll member of the Twilight's Ha...,"[Chromatus, Josah, Kirygosa]"
1970,Zuzak,"Zuzak was assigned to Ziya's plundersquad, whi...",[Ziya]


In [35]:
with open('df_text.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)